# 전기차 충전소 도로명 주소 -> 좌표계 전환

In [1]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json
import pickle

In [2]:
df = pd.read_excel('/Users/jacob/Desktop/VS_Code_Repository/DeepLearning_Alpaco/빅콘테스트2022/Raw data/충전소 리스트.xlsx',header=2)
df=df[df['시군구']=='용인시 ']

In [3]:
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json
import numpy as np
from tqdm import tqdm

#naver map api key
client_id = 'cid';    # 본인이 할당받은 ID 입력
client_pw = 'cpw';    # 본인이 할당받은 Secret 입력

api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

# 주소 목록 파일 (.csv)
data = df

# 네이버 지도 API 이용해서 위경도 찾기
geo_coordi = []     
resp=''
que=''
sent=''
for i,row in tqdm(data.iterrows(),total=data.shape[0]):
    add = row['주소']
    add_urlenc = parse.quote(add)  
    url = api_url + add_urlenc
    que=url
    sent=add
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        # print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)   # json
            resp=response_body
            if response_body['status']=='INVALID_REQUEST' : continue
            if response_body['addresses'] == [] :
                # print("'result' not exist!")
                latitude = None
                longitude = None
            else:
                df.at[i,'위도']= response_body['addresses'][0]['y']
                df.at[i,'경도']= response_body['addresses'][0]['x']
                # print("Success!")
        else:
            print('Response error code : %d' % rescode)
            latitude = None
            longitude = None

100%|██████████| 4199/4199 [10:59<00:00,  6.36it/s]  


In [4]:
df.to_csv('/Users/jacob/Desktop/VS_Code_Repository/DeepLearning_Alpaco/빅콘테스트2022/Raw data/echarge_yongin_naver_xy.csv',encoding='utf-8',index=False)

In [10]:
df.dropna(subset=['위도'],inplace=True)
df.drop_duplicates(subset='충전소',inplace=True)
# 공공 충전시설만 추출(수작업 필터링)
with open('/Users/jacob/Desktop/VS_Code_Repository/DeepLearning_Alpaco/빅콘테스트2022/Bigcon_data/public.txt','rt') as f:
    t = [i.strip() for i in f.read().split(',')]
df= df[df['충전소'].isin(t)]
df.to_csv()

In [12]:
df.to_csv('/Users/jacob/Desktop/VS_Code_Repository/DeepLearning_Alpaco/빅콘테스트2022/Raw data/echarge_yongin_public_naver_xy.csv',encoding='utf-8',index=False)